In [1]:
#Add parent and grandparent folder to sys.path and get variable name
import os
import sys
import inspect

parentdir = os.path.abspath('..')
grandparentdir=os.path.dirname(parentdir)
ggpdir=os.path.dirname(grandparentdir)
sys.path.insert(0, parentdir)
sys.path.insert(0, grandparentdir)
sys.path.insert(0, ggpdir)

var_name=os.path.basename(parentdir)

var_name

'consumpinit_scoe_gj_per_hh_residential_elec_appliances'

In [2]:
import pandas as pd
import numpy as np

df_countries=pd.read_csv("../row_data/Countries_ISO3.csv")
df_countries

,Country Name,ISO3
0,Aruba,ABW
1,Africa Eastern and Southern,AFE
2,Afghanistan,AFG
3,Africa Western and Central,AFW
4,Angola,AGO
...,...,...
261,Kosovo,XKX
262,"Yemen, Rep.",YEM
263,South Africa,ZAF
264,Zambia,ZMB


In [3]:
df_popur_hist = pd.read_csv(ggpdir+
        "/SocioEconomic/population_gnrl_urban/input_to_sisepuede/historical/population_gnrl_urban.csv") 
df_popur_proj = pd.read_csv(ggpdir+
        "/SocioEconomic/population_gnrl_urban/input_to_sisepuede/projected/population_gnrl_urban.csv")

df_popru_hist = pd.read_csv(ggpdir+
        "/SocioEconomic/population_gnrl_rural/input_to_sisepuede/historical/population_gnrl_rural.csv") 
df_popru_proj = pd.read_csv(ggpdir+
        "/SocioEconomic/population_gnrl_rural/input_to_sisepuede/projected/population_gnrl_rural.csv")

df_occ_hist = pd.read_csv(ggpdir+
        "/SocioEconomic/occrateinit_gnrl_occupancy/input_to_sisepuede/historical/occrateinit_gnrl_occupancy.csv")
df_occ_proj = pd.read_csv(ggpdir+
        "/SocioEconomic/occrateinit_gnrl_occupancy/input_to_sisepuede/projected/occrateinit_gnrl_occupancy.csv")

# hh = round((population_rurual + population_urban)/occ_rate)

#countries=df_countries['Country Name'].values
set1 = set(df_popur_hist['iso_code3'].values)
set2 = set(df_popru_hist['iso_code3'].values)
set3 = set(df_occ_hist['iso_code3'].values)

common_set = set1.intersection(set2, set3)
common_elements = sorted(list(common_set))
ISO3=common_elements

#countries=np.unique(df_countries['Nation'].values)
years_hist=range(max(min(zip(df_popur_hist['Year'].values,
                             df_popru_hist['Year'].values,
                             df_occ_hist['Year'].values))),
                min(max(zip(df_popur_hist['Year'].values,
                            df_popru_hist['Year'].values,
                            df_occ_hist['Year'].values)))
                )
coun_year=[]
for iso3 in ISO3:
    count = df_countries.loc[df_countries['ISO3']==iso3]['Country Name'].values[0]
    for year in years_hist:
        pop_ru = df_popru_hist.loc[(df_popru_hist['iso_code3'] == iso3) & 
                        (df_popru_hist['Year']== year), 'population_gnrl_rural'].values[0]
        
        pop_ur = df_popur_hist.loc[(df_popur_hist['iso_code3'] == iso3) & 
                        (df_popur_hist['Year']== year), 'population_gnrl_urban'].values[0]
        
        occ_rate = df_occ_hist.loc[(df_occ_hist['iso_code3'] == iso3) & 
                        (df_occ_hist['Year']== year), 'occrateinit_gnrl_occupancy'].values[0]
        
        hh = round((pop_ru+pop_ur)/occ_rate)
        coun_year.append([count,iso3,year,hh])
        
df_hh_hist = pd.DataFrame(coun_year, columns=['Nation','iso_code3', 'Year','hh'])
df_hh_hist

,Nation,iso_code3,Year,hh
0,Aruba,ABW,2011,35311
1,Aruba,ABW,2012,35490
2,Aruba,ABW,2013,35697
3,Aruba,ABW,2014,35909
4,Aruba,ABW,2015,36103
...,...,...,...,...
1633,Zimbabwe,ZWE,2015,3282501
1634,Zimbabwe,ZWE,2016,3333753
1635,Zimbabwe,ZWE,2017,3382763
1636,Zimbabwe,ZWE,2018,3430811


In [4]:
years_proj=range(max(min(zip(df_popur_proj['Year'].values,
                             df_popru_proj['Year'].values,
                             df_occ_proj['Year'].values))),
                min(max(zip(df_popur_proj['Year'].values,
                            df_popru_proj['Year'].values,
                            df_occ_proj['Year'].values)))+1
                )

years_proj=range(2020,
                min(max(zip(df_popur_proj['Year'].values,
                            df_popru_proj['Year'].values,
                            df_occ_proj['Year'].values)))+1
                )
coun_year=[]
for iso3 in ISO3:
    count = df_countries.loc[df_countries['ISO3']==iso3]['Country Name'].values[0]
    for year in years_proj:
        try:
            pop_ru = df_popru_proj.loc[(df_popru_proj['iso_code3'] == iso3) & 
                            (df_popru_proj['Year']== year), 'population_gnrl_rural'].values[0]
        except:
            pop_ru = df_popru_hist.loc[(df_popru_hist['iso_code3'] == iso3) & 
                            (df_popru_hist['Year']== year), 'population_gnrl_rural'].values[0]
        try:
            pop_ur = df_popur_proj.loc[(df_popur_proj['iso_code3'] == iso3) & 
                            (df_popur_proj['Year']== year), 'population_gnrl_urban'].values[0]
        except:
            pop_ur = df_popur_hist.loc[(df_popur_hist['iso_code3'] == iso3) & 
                        (df_popur_hist['Year']== year), 'population_gnrl_urban'].values[0]
        try:
            occ_rate = df_occ_proj.loc[(df_occ_proj['iso_code3'] == iso3) & 
                            (df_occ_proj['Year']== year), 'occrateinit_gnrl_occupancy'].values[0]
        except:
            occ_rate = df_occ_hist.loc[(df_occ_hist['iso_code3'] == iso3) & 
                        (df_occ_hist['Year']== year), 'occrateinit_gnrl_occupancy'].values[0]
        hh = round((pop_ru+pop_ur)/occ_rate)
        coun_year.append([count,iso3,year,hh])
        
df_hh_proj = pd.DataFrame(coun_year, columns=['Nation','iso_code3', 'Year','hh'])
df_hh_proj

,Nation,iso_code3,Year,hh
0,Aruba,ABW,2020,43340
1,Aruba,ABW,2021,43804
2,Aruba,ABW,2022,44268
3,Aruba,ABW,2023,44732
4,Aruba,ABW,2024,45196
...,...,...,...,...
5637,Zimbabwe,ZWE,2046,3140103
5638,Zimbabwe,ZWE,2047,3134648
5639,Zimbabwe,ZWE,2048,3129192
5640,Zimbabwe,ZWE,2049,3123737


In [5]:
df_hh_proj[df_hh_proj['Nation'].str.startswith('Tur')]

,Nation,iso_code3,Year,hh
4991,Turkmenistan,TKM,2020,1093411
4992,Turkmenistan,TKM,2021,1102406
4993,Turkmenistan,TKM,2022,1111400
4994,Turkmenistan,TKM,2023,1120394
4995,Turkmenistan,TKM,2024,1129388
...,...,...,...,...
5172,Turkey,TUR,2046,22677181
5173,Turkey,TUR,2047,22740658
5174,Turkey,TUR,2048,22804135
5175,Turkey,TUR,2049,22867612


In [6]:
# df_WEBH2021=pd.read_excel('../row_data/WorldEnergyBalancesHighlights2021.xlsx',sheet_name=3)

df_energy=pd.read_excel('../row_data/IEA - Energy Efficiency Indicators database - Extended (December 2022).xlsb',
                        engine='pyxlsb',sheet_name='Residential - Energy',skiprows=[0])
df_energy=df_energy.replace('..',0)
df_energy

,Country,End use,Product,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Australia,Total Residential,Oil and oil products (PJ) ...,14.63,14.35,14.44,14.97,13.57,12.07,14.46,...,15.87,15.58,15.87,15.68,15.30,16.70,14.65,14.74,15.20,14.73
1,Australia,Total Residential,Gas (PJ) ...,104.84,107.69,110.50,110.61,110.85,113.71,117.22,...,131.16,133.55,136.95,140.58,143.29,146.38,146.15,145.92,148.58,154.10
2,Australia,Total Residential,Coal and coal products (PJ) ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Australia,Total Residential,Biofuels and waste (PJ) ...,80.60,79.50,76.34,73.20,70.60,67.80,65.58,...,52.80,52.80,52.80,51.77,49.80,49.22,48.80,48.20,47.60,47.10
4,Australia,Total Residential,Heat (PJ) ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4408,Uruguay,Non-specified,Biofuels and waste (PJ) ...,12.69,12.69,12.66,12.66,12.66,12.66,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.31,0.00
4409,Uruguay,Non-specified,Heat (PJ) ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4410,Uruguay,Non-specified,Electricity (PJ) ...,10.42,10.41,10.07,9.40,9.45,9.70,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.65,0.00
4411,Uruguay,Non-specified,Other sources (PJ) ...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
df_energy_totalproduct=df_energy[df_energy['Product'].str.startswith('Total final energy use (PJ)')]
df_energy_totalproduct[df_energy_totalproduct['Country']=='Uruguay']

,Country,End use,Product,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
4343,Uruguay,Total Residential,Total final energy use (PJ) ...,30.3,29.79,29.19,28.0,27.73,27.92,29.45,...,31.48,31.88,32.27,32.60,33.09,34.18,33.39,33.86,35.22,0.0
4351,Uruguay,Residential space heating,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,9.89,...,10.90,12.02,11.98,12.16,12.44,12.47,12.57,12.63,0.00,0.0
4355,Uruguay,Residential space cooling,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,0.25,...,0.58,0.72,0.85,1.08,1.22,1.27,1.39,1.42,0.00,0.0
4363,Uruguay,Residential water heating,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,5.14,...,5.40,5.19,5.52,5.74,5.85,5.91,5.98,6.08,0.00,0.0
4371,Uruguay,Residential cooking,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,8.28,...,6.89,5.80,5.86,5.76,5.56,5.56,5.40,5.39,0.00,0.0
4374,Uruguay,Residential lighting,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4377,Uruguay,Residential appliances,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,5.89,...,7.71,8.15,8.06,7.85,8.02,8.96,8.06,8.34,0.00,0.0
4380,Uruguay,Refrigerators,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4383,Uruguay,Freezers,Total final energy use (PJ) ...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4386,Uruguay,Refrigerator/Freezer combinations,Total final energy use (PJ),0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [8]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
Nations = df_energy_totalproduct['Country'].unique()
coun_year=[]
for count in Nations:
        for year in years_hist:
            coun_year.append([count,year])
            
merged_df=pd.DataFrame(coun_year, columns=['Nation', 'Year'])

for end_use in [eu for eu in df_energy_totalproduct['End use'].unique() if eu !='Total Residential']:
    df_enduse=df_energy_totalproduct[df_energy_totalproduct['End use']==end_use]
    nation_enduse = df_enduse['Country'].values
#     years_hist = range(min(df_energy_totalproduct.columns),2020+1)

    c_y_enduse = []
    for c in Nations:
        for y in years_hist:

            end_use_num = df_enduse.loc[(df_enduse['Country']==c)][y].values[0]
            
            total=df_energy_totalproduct.loc[(df_energy_totalproduct['Country']==c)][y].values[0]
            
#             total=df_enduse_complete[(df_enduse_complete['Nation']==c)&
#                             (df_enduse_complete['Year']==y)]['Total Residential Total (PJ)'].values[0]
            if total!=0:
                end_use_frac= end_use_num/total
            else:
                end_use_frac=0

            if is_number(end_use_num):
                end_use_num=float(end_use_num)

#                 c_y_enduse.append([c,y,end_use_num,end_use_frac])

#     df_enduse = pd.DataFrame(c_y_enduse,columns=['Nation', 'Year',end_use+'Total (PJ)',end_use+'_Fraction'])
                c_y_enduse.append([c,y,end_use_num])

    df_enduse = pd.DataFrame(c_y_enduse,columns=['Nation', 'Year',end_use])

#     nation_mean=[]
#     for nation in df_enduse['Nation'].unique():
#         values=df_enduse.loc[df_enduse['Nation']==nation][end_use+'Total (PJ)'].values
#         nation_mean.append([nation,np.mean(values)])
#     mean_df=pd.DataFrame(nation_mean,columns=['Nation','end_use_mean'])

#     # Fill holes in present nations with the mean value
    
#     for count in Nations:
#         for year in years_hist:
#             PJ=df_enduse.loc[(df_enduse['Nation'] ==count) & 
#                             (df_enduse['Year']== year), end_use+'Total (PJ)'].values[0]
#             if PJ==0:
#                 PJ=mean_df.loc[mean_df['Nation'] ==count,'end_use_mean'].values[0]
#                 df_enduse.loc[(df_enduse['Nation'] ==count) & 
#                             (df_enduse['Year']== year), end_use+'Total (PJ)']=PJ
#    consumpinit_scoe_gj_per_hh_residential_elec_appliances : [all but (residential space heating +residential cooking+ clothes dryers)]
#    consumpinit_scoe_gj_per_hh_residential_heat_energy : [residential space heating +residential cooking+ clothes dryers]

    merged_df=pd.merge(merged_df, df_enduse, on=['Nation', 'Year'])
merged_df

,Nation,Year,Residential space heating,Residential space cooling,Residential water heating,Residential cooking,Residential lighting,Residential appliances,Refrigerators,Freezers,Refrigerator/Freezer combinations,Dish washers,Clothes washers,Clothes dryers,Televisions,Personal computers,Other appliances,Non-specified
0,Australia,2011,159.30,19.13,104.59,21.07,30.39,103.98,0.0,6.28,26.52,3.35,2.97,1.19,22.17,8.73,32.76,0.00
1,Australia,2012,159.83,19.47,105.69,21.00,29.21,102.30,0.0,6.17,26.18,3.39,3.08,1.20,20.96,8.21,33.12,0.00
2,Australia,2013,162.49,19.83,105.25,21.51,28.55,101.17,0.0,6.12,26.13,3.47,3.20,1.22,19.48,7.61,33.93,0.00
3,Australia,2014,163.35,19.32,103.05,21.67,26.82,97.15,0.0,5.88,25.30,3.43,3.24,1.21,17.32,6.86,33.90,0.00
4,Australia,2015,162.72,20.29,107.39,22.29,26.48,99.76,0.0,6.04,26.24,3.63,3.50,1.26,16.44,6.68,35.96,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Uruguay,2015,12.44,1.22,5.85,5.56,0.00,8.02,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.02,0.00
509,Uruguay,2016,12.47,1.27,5.91,5.56,0.00,8.96,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.96,0.00
510,Uruguay,2017,12.57,1.39,5.98,5.40,0.00,8.06,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.06,0.00
511,Uruguay,2018,12.63,1.42,6.08,5.39,0.00,8.34,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.34,0.00


In [9]:
df_energy_totalproduct['End use'].unique()

array(['Total Residential', 'Residential space heating',
       'Residential space cooling', 'Residential water heating',
       'Residential cooking', 'Residential lighting',
       'Residential appliances', 'Refrigerators', 'Freezers',
       'Refrigerator/Freezer combinations', 'Dish washers',
       'Clothes washers', 'Clothes dryers', 'Televisions',
       'Personal computers', 'Other appliances', 'Non-specified'],
      dtype=object)

In [10]:
heat_enduse=['Residential space heating','Residential cooking','Clothes dryers']
merged_df['Heat_Total_PJ']=0
merged_df['Elect_Total_PJ']=0
for c in Nations:
    for y in years_hist:
        heat_vect=[]
        electric_vect=[]
        df_c_y=merged_df.loc[(merged_df['Nation']==c)&(merged_df['Year']==y)]
        for enduse in [eu for eu in df_energy_totalproduct['End use'].unique() if eu !='Total Residential']:
            if enduse in heat_enduse:
                heat_val=df_c_y[enduse].values[0]
                heat_vect.append(heat_val)
            else:
                elect_val=df_c_y[enduse].values[0]
                electric_vect.append(elect_val)
                
        merged_df.loc[(merged_df['Nation']==c)&(merged_df['Year']==y),'Elect_Total_PJ']=sum(electric_vect)
        merged_df.loc[(merged_df['Nation']==c)&(merged_df['Year']==y),'Heat_Total_PJ']=sum(heat_vect)
merged_df

,Nation,Year,Residential space heating,Residential space cooling,Residential water heating,Residential cooking,Residential lighting,Residential appliances,Refrigerators,Freezers,Refrigerator/Freezer combinations,Dish washers,Clothes washers,Clothes dryers,Televisions,Personal computers,Other appliances,Non-specified,Heat_Total_PJ,Elect_Total_PJ
0,Australia,2011,159.30,19.13,104.59,21.07,30.39,103.98,0.0,6.28,26.52,3.35,2.97,1.19,22.17,8.73,32.76,0.00,181.56,360.87
1,Australia,2012,159.83,19.47,105.69,21.00,29.21,102.30,0.0,6.17,26.18,3.39,3.08,1.20,20.96,8.21,33.12,0.00,182.03,357.78
2,Australia,2013,162.49,19.83,105.25,21.51,28.55,101.17,0.0,6.12,26.13,3.47,3.20,1.22,19.48,7.61,33.93,0.00,185.22,354.74
3,Australia,2014,163.35,19.32,103.05,21.67,26.82,97.15,0.0,5.88,25.30,3.43,3.24,1.21,17.32,6.86,33.90,0.00,186.23,342.27
4,Australia,2015,162.72,20.29,107.39,22.29,26.48,99.76,0.0,6.04,26.24,3.63,3.50,1.26,16.44,6.68,35.96,0.00,186.27,352.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Uruguay,2015,12.44,1.22,5.85,5.56,0.00,8.02,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.02,0.00,18.00,23.11
509,Uruguay,2016,12.47,1.27,5.91,5.56,0.00,8.96,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.96,0.00,18.03,25.10
510,Uruguay,2017,12.57,1.39,5.98,5.40,0.00,8.06,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.06,0.00,17.97,23.49
511,Uruguay,2018,12.63,1.42,6.08,5.39,0.00,8.34,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.34,0.00,18.02,24.18


In [11]:
#Socioeconomic varibles

#GDP
df_gdp_hist = pd.read_csv(ggpdir+'/Socioeconomic/gdp_mmm_usd/input_to_sisepuede/historical/gdp_mmm_usd.csv')
df_gdp_proj = pd.read_csv(ggpdir+'/Socioeconomic/gdp_mmm_usd/input_to_sisepuede/projected/gdp_mmm_usd.csv')

nation_gdp = sorted(set(df_gdp_proj['Nation'].values))

years_all = range(min(df_gdp_hist['Year'].values),max(df_gdp_proj['Year'].values)+1)
c_y_gdp = []
for c in nation_gdp:
    for y in years_all:
    
        try:
            gdp = df_gdp_hist.loc[(df_gdp_hist['Nation']==c) & (df_gdp_hist['Year']==y),]['gdp_mmm_usd'].values[0]
            
        except:
            gdp = df_gdp_proj.loc[(df_gdp_proj['Nation']==c) & (df_gdp_proj['Year']==y)]['gdp_mmm_usd'].values[0]
            
        c_y_gdp.append([c,y,gdp])

df_gdp_all = pd.DataFrame(c_y_gdp,columns=['Nation', 'Year', 'gdp_mmm_usd'])



#Rural population
df_popru_hist = pd.read_csv(ggpdir+
        '/Socioeconomic/population_gnrl_rural/input_to_sisepuede/historical/population_gnrl_rural.csv')
df_popru_proj = pd.read_csv(ggpdir+
        '/Socioeconomic/population_gnrl_rural/input_to_sisepuede/projected/population_gnrl_rural.csv')

#Urban population
df_popur_hist = pd.read_csv(ggpdir+
        '/Socioeconomic/population_gnrl_urban/input_to_sisepuede/historical/population_gnrl_urban.csv')
df_popur_proj = pd.read_csv(ggpdir+
        '/Socioeconomic/population_gnrl_urban/input_to_sisepuede/projected/population_gnrl_urban.csv')

nation_pop = sorted(set(df_popur_proj['Nation'].values))

years_all = range(min(df_popur_hist['Year'].values),max(df_popur_proj['Year'].values)+1)
c_y_pop = []
for c in nation_pop:
    for y in years_all:
    
        try:
            pop_ur = df_popur_hist.loc[(df_popur_hist['Nation']==c) &
                                       (df_popur_hist['Year']==y),]['population_gnrl_urban'].values[0]
            pop_ru = df_popru_hist.loc[(df_popru_hist['Nation']==c) &
                                       (df_popru_hist['Year']==y),]['population_gnrl_rural'].values[0]
            
        except:
            pop_ur = df_popur_proj.loc[(df_popur_proj['Nation']==c) &
                                       (df_popur_proj['Year']==y),]['population_gnrl_urban'].values[0]
            pop_ru = df_popru_proj.loc[(df_popru_proj['Nation']==c) &
                                       (df_popru_proj['Year']==y),]['population_gnrl_rural'].values[0]
            
        c_y_pop.append([c,y,pop_ur,pop_ru])

df_pop_all = pd.DataFrame(c_y_pop,columns=['Nation', 'Year', 'population_gnrl_urban','population_gnrl_rural'])

#All Together
nation_pop_gdp = sorted(set(nation_pop).intersection(set(nation_gdp)))
years_all = range(min(df_popur_hist['Year'].values),max(df_popur_proj['Year'].values)+1)
c_y_pop_gdp = []
for c in nation_pop_gdp:
    iso3 = df_gdp_hist.loc[df_gdp_hist['Nation']==c]['iso_code3'].values[0]
    for y in years_all:
        pop_ur = df_pop_all.loc[(df_pop_all['Nation']==c) &
                                       (df_pop_all['Year']==y),]['population_gnrl_urban'].values[0]
        
        pop_ru = df_pop_all.loc[(df_pop_all['Nation']==c) &
                                       (df_pop_all['Year']==y),]['population_gnrl_rural'].values[0]
        
        gdp = df_gdp_all.loc[(df_gdp_all['Nation']==c) & (df_gdp_all['Year']==y)]['gdp_mmm_usd'].values[0]
        
        c_y_pop_gdp.append([c,iso3,y,pop_ur,pop_ru,gdp])
df_pop_gdp_all = pd.DataFrame(c_y_pop_gdp,columns=['Nation','iso_code3', 'Year',
                                                   'population_gnrl_urban',
                                                   'population_gnrl_rural',
                                                   'gdp_mmm_usd'])

df_pop_gdp_all['population_gnrl_urban(millions)']=df_pop_gdp_all.apply(lambda x: x['population_gnrl_urban']*1e-6, axis=1)
df_pop_gdp_all['population_gnrl_rural(millions)']=df_pop_gdp_all.apply(lambda x: x['population_gnrl_rural']*1e-6, axis=1)
df_pop_gdp_all

,Nation,iso_code3,Year,population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions)
0,Afghanistan,AFG,2011,7.212518e+06,2.290489e+07,35.369178,7.212518,22.904893
1,Afghanistan,AFG,2012,7.528589e+06,2.363279e+07,38.085386,7.528589,23.632789
2,Afghanistan,AFG,2013,7.865068e+06,2.440452e+07,40.801594,7.865068,24.404524
3,Afghanistan,AFG,2014,8.204880e+06,2.516592e+07,43.517802,8.204880,25.165924
4,Afghanistan,AFG,2015,8.535606e+06,2.587800e+07,46.234010,8.535606,25.877997
...,...,...,...,...,...,...,...,...
7275,Zimbabwe,ZWE,2046,5.352032e+06,7.863317e+06,24.382002,5.352032,7.863317
7276,Zimbabwe,ZWE,2047,5.384492e+06,7.807897e+06,26.002852,5.384492,7.807897
7277,Zimbabwe,ZWE,2048,5.416953e+06,7.752477e+06,27.623703,5.416953,7.752477
7278,Zimbabwe,ZWE,2049,5.449414e+06,7.697057e+06,29.244553,5.449414,7.697057


In [12]:
joined_df = df_pop_gdp_all.merge(merged_df[['Nation','Year','Heat_Total_PJ', 'Elect_Total_PJ']], on=['Nation','Year'])

# joined_df=joined_df.merge(df_hh_hist)

joined_df['population_gnrl_urban(millions)']=joined_df.apply(lambda x: x['population_gnrl_urban']*1e-6, axis=1)
joined_df['population_gnrl_rural(millions)']=joined_df.apply(lambda x: x['population_gnrl_rural']*1e-6, axis=1)

joined_df


,Nation,iso_code3,Year,population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions),Heat_Total_PJ,Elect_Total_PJ
0,Albania,ALB,2011,1.546929e+06,1.358266e+06,21.159910,1.546929,1.358266,12.88,9.76
1,Albania,ALB,2012,1.575788e+06,1.324613e+06,21.865577,1.575788,1.324613,13.45,10.17
2,Albania,ALB,2013,1.603505e+06,1.291587e+06,22.571244,1.603505,1.291587,14.51,12.99
3,Albania,ALB,2014,1.630119e+06,1.258985e+06,23.276911,1.630119,1.258985,14.33,11.76
4,Albania,ALB,2015,1.654503e+06,1.226200e+06,23.982578,1.654503,1.226200,13.49,11.24
...,...,...,...,...,...,...,...,...,...,...
445,Uruguay,URY,2015,3.242948e+06,1.690652e+05,51.626567,3.242948,0.169065,18.00,23.11
446,Uruguay,URY,2016,3.257863e+06,1.662762e+05,52.159029,3.257863,0.166276,18.03,25.10
447,Uruguay,URY,2017,3.273061e+06,1.635843e+05,52.691490,3.273061,0.163584,17.97,23.49
448,Uruguay,URY,2018,3.288346e+06,1.609439e+05,53.223952,3.288346,0.160944,18.02,24.18


In [13]:
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize

X=joined_df[['population_gnrl_urban(millions)','population_gnrl_rural(millions)','gdp_mmm_usd']]
Y_elect=joined_df['Elect_Total_PJ']
Y_heat=joined_df['Heat_Total_PJ']


linear_model_heat = LinearRegression()
linear_model_elect = LinearRegression()

linear_model_heat.fit(X,Y_heat)
linear_model_elect.fit(X,Y_elect)

c_y_iso3_h_e=[]
years_all=range(2011,2051)

for c in sorted(set(df_hh_hist['Nation'].unique()).intersection(df_pop_gdp_all['Nation'].unique())):
    iso3 = df_countries.loc[df_countries['Country Name']==c]['ISO3'].values[0]
    for y in years_all:
        if (c,y) in zip(joined_df['Nation'],joined_df['Year']):
            
            h=joined_df[(joined_df['Nation']==c)&(joined_df['Year']==y)]['Heat_Total_PJ'].values[0]*1e6
            E=joined_df[(joined_df['Nation']==c)&(joined_df['Year']==y)]['Elect_Total_PJ'].values[0]*1e6
            
        else:
            X_test = df_pop_gdp_all.loc[(df_pop_gdp_all['Nation'] == c) & 
                                (df_pop_gdp_all['Year']== y)][['population_gnrl_urban(millions)',
                                                                  'population_gnrl_rural(millions)',
                                                                  'gdp_mmm_usd']]
            h =linear_model_heat.predict(X_test)[0]*1e6
            E =linear_model_elect.predict(X_test)[0]*1e6
            
            if h<0:
                h=0
            if E<0:
                E=0
        
        c_y_iso3_h_e.append([c,iso3,y,h,E])

total_heat_elect_df=pd.DataFrame(c_y_iso3_h_e,columns=['Nation','iso_code3','Year','Heat_Total_GJ','Elect_Total_GJ'])
total_heat_elect_df

,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08
...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06


In [14]:
df_hh_heat_elect_hist=df_hh_hist.merge(total_heat_elect_df,on=['Nation','iso_code3','Year'])
df_hh_heat_elect_hist

,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ
0,Aruba,ABW,2011,35311,2.656550e+07,0.000000e+00
1,Aruba,ABW,2012,35490,2.666319e+07,0.000000e+00
2,Aruba,ABW,2013,35697,2.676111e+07,0.000000e+00
3,Aruba,ABW,2014,35909,2.685855e+07,0.000000e+00
4,Aruba,ABW,2015,36103,2.695507e+07,0.000000e+00
...,...,...,...,...,...,...
1624,Zimbabwe,ZWE,2015,3282501,1.888983e+08,1.530004e+07
1625,Zimbabwe,ZWE,2016,3333753,1.917904e+08,1.641311e+07
1626,Zimbabwe,ZWE,2017,3382763,1.944553e+08,1.743358e+07
1627,Zimbabwe,ZWE,2018,3430811,1.969458e+08,1.838103e+07


In [15]:
df_hh_heat_elect_proj=df_hh_proj.merge(total_heat_elect_df,on=['Nation','iso_code3','Year'])
df_hh_heat_elect_proj

,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ
0,Aruba,ABW,2020,43340,2.686656e+07,0.000000e+00
1,Aruba,ABW,2021,43804,2.704094e+07,0.000000e+00
2,Aruba,ABW,2022,44268,2.706157e+07,0.000000e+00
3,Aruba,ABW,2023,44732,2.708220e+07,0.000000e+00
4,Aruba,ABW,2024,45196,2.710283e+07,0.000000e+00
...,...,...,...,...,...,...
5606,Zimbabwe,ZWE,2046,3140103,1.577604e+08,8.833231e+06
5607,Zimbabwe,ZWE,2047,3134648,1.570312e+08,9.143963e+06
5608,Zimbabwe,ZWE,2048,3129192,1.563020e+08,9.454694e+06
5609,Zimbabwe,ZWE,2049,3123737,1.555727e+08,9.765425e+06


In [16]:
df_hh_heat_elect_hist['consumpinit_scoe_gj_per_hh_residential_elec_appliances'] = df_hh_heat_elect_hist.apply(lambda row: row['Elect_Total_GJ'] / (row['hh'] ), axis=1)
df_hh_heat_elect_hist['consumpinit_scoe_gj_per_hh_residential_heat_energy'] = df_hh_heat_elect_hist.apply(lambda row: row['Heat_Total_GJ'] / (row['hh'] ), axis=1)
df_hh_heat_elect_hist

,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ,consumpinit_scoe_gj_per_hh_residential_elec_appliances,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2011,35311,2.656550e+07,0.000000e+00,0.000000,752.329330
1,Aruba,ABW,2012,35490,2.666319e+07,0.000000e+00,0.000000,751.287485
2,Aruba,ABW,2013,35697,2.676111e+07,0.000000e+00,0.000000,749.673942
3,Aruba,ABW,2014,35909,2.685855e+07,0.000000e+00,0.000000,747.961548
4,Aruba,ABW,2015,36103,2.695507e+07,0.000000e+00,0.000000,746.615669
...,...,...,...,...,...,...,...,...
1624,Zimbabwe,ZWE,2015,3282501,1.888983e+08,1.530004e+07,4.661093,57.547081
1625,Zimbabwe,ZWE,2016,3333753,1.917904e+08,1.641311e+07,4.923313,57.529891
1626,Zimbabwe,ZWE,2017,3382763,1.944553e+08,1.743358e+07,5.153652,57.484153
1627,Zimbabwe,ZWE,2018,3430811,1.969458e+08,1.838103e+07,5.357634,57.405015


In [17]:
df_hist_heat=df_hh_heat_elect_hist[['Nation','iso_code3','Year','consumpinit_scoe_gj_per_hh_residential_heat_energy']]
df_hist_heat

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2011,752.329330
1,Aruba,ABW,2012,751.287485
2,Aruba,ABW,2013,749.673942
3,Aruba,ABW,2014,747.961548
4,Aruba,ABW,2015,746.615669
...,...,...,...,...
1624,Zimbabwe,ZWE,2015,57.547081
1625,Zimbabwe,ZWE,2016,57.529891
1626,Zimbabwe,ZWE,2017,57.484153
1627,Zimbabwe,ZWE,2018,57.405015


In [18]:
df_hist_elec=df_hh_heat_elect_hist[['Nation','iso_code3','Year','consumpinit_scoe_gj_per_hh_residential_elec_appliances']]
df_hist_elec

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_elec_appliances
0,Aruba,ABW,2011,0.000000
1,Aruba,ABW,2012,0.000000
2,Aruba,ABW,2013,0.000000
3,Aruba,ABW,2014,0.000000
4,Aruba,ABW,2015,0.000000
...,...,...,...,...
1624,Zimbabwe,ZWE,2015,4.661093
1625,Zimbabwe,ZWE,2016,4.923313
1626,Zimbabwe,ZWE,2017,5.153652
1627,Zimbabwe,ZWE,2018,5.357634


In [19]:
df_hh_heat_elect_proj['consumpinit_scoe_gj_per_hh_residential_elec_appliances'] = df_hh_heat_elect_proj.apply(lambda row: row['Elect_Total_GJ'] / (row['hh'] ), axis=1)
df_hh_heat_elect_proj['consumpinit_scoe_gj_per_hh_residential_heat_energy'] = df_hh_heat_elect_proj.apply(lambda row: row['Heat_Total_GJ'] / (row['hh'] ), axis=1)
df_hh_heat_elect_proj


,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ,consumpinit_scoe_gj_per_hh_residential_elec_appliances,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2020,43340,2.686656e+07,0.000000e+00,0.000000,619.902089
1,Aruba,ABW,2021,43804,2.704094e+07,0.000000e+00,0.000000,617.316795
2,Aruba,ABW,2022,44268,2.706157e+07,0.000000e+00,0.000000,611.312275
3,Aruba,ABW,2023,44732,2.708220e+07,0.000000e+00,0.000000,605.432324
4,Aruba,ABW,2024,45196,2.710283e+07,0.000000e+00,0.000000,599.673104
...,...,...,...,...,...,...,...,...
5606,Zimbabwe,ZWE,2046,3140103,1.577604e+08,8.833231e+06,2.813039,50.240531
5607,Zimbabwe,ZWE,2047,3134648,1.570312e+08,9.143963e+06,2.917062,50.095323
5608,Zimbabwe,ZWE,2048,3129192,1.563020e+08,9.454694e+06,3.021449,49.949625
5609,Zimbabwe,ZWE,2049,3123737,1.555727e+08,9.765425e+06,3.126200,49.803402


In [20]:
df_proj_elec=df_hh_heat_elect_proj[['Nation','iso_code3','Year','consumpinit_scoe_gj_per_hh_residential_elec_appliances']]
df_proj_elec


,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_elec_appliances
0,Aruba,ABW,2020,0.000000
1,Aruba,ABW,2021,0.000000
2,Aruba,ABW,2022,0.000000
3,Aruba,ABW,2023,0.000000
4,Aruba,ABW,2024,0.000000
...,...,...,...,...
5606,Zimbabwe,ZWE,2046,2.813039
5607,Zimbabwe,ZWE,2047,2.917062
5608,Zimbabwe,ZWE,2048,3.021449
5609,Zimbabwe,ZWE,2049,3.126200


In [21]:
df_proj_heat=df_hh_heat_elect_proj[['Nation','iso_code3','Year','consumpinit_scoe_gj_per_hh_residential_heat_energy']]
df_proj_heat

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2020,619.902089
1,Aruba,ABW,2021,617.316795
2,Aruba,ABW,2022,611.312275
3,Aruba,ABW,2023,605.432324
4,Aruba,ABW,2024,599.673104
...,...,...,...,...
5606,Zimbabwe,ZWE,2046,50.240531
5607,Zimbabwe,ZWE,2047,50.095323
5608,Zimbabwe,ZWE,2048,49.949625
5609,Zimbabwe,ZWE,2049,49.803402


In [22]:
df_hist_heat.to_csv(grandparentdir+'/consumpinit_scoe_gj_per_hh_residential_heat_energy/'+
                   '/input_to_sisepuede/historical/consumpinit_scoe_gj_per_hh_residential_heat_energy.csv')
df_hist_elec.to_csv(grandparentdir+'/consumpinit_scoe_gj_per_hh_residential_elec_appliances/'+
                   '/input_to_sisepuede/historical/consumpinit_scoe_gj_per_hh_residential_elec_appliances.csv')
df_proj_heat.to_csv(grandparentdir+'/consumpinit_scoe_gj_per_hh_residential_heat_energy/'+
                   '/input_to_sisepuede/projected/consumpinit_scoe_gj_per_hh_residential_heat_energy.csv')
df_proj_elec.to_csv(grandparentdir+'/consumpinit_scoe_gj_per_hh_residential_elec_appliances/'+
                   '/input_to_sisepuede/projected/consumpinit_scoe_gj_per_hh_residential_elec_appliances.csv')

In [23]:
df_hh_heat_elect_all=pd.concat([df_hh_heat_elect_hist,df_hh_heat_elect_proj])
df_hh_heat_elect_all.to_csv('../row_data/hh_heat_elec_gj_all.csv')
df_hh_heat_elect_all

,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ,consumpinit_scoe_gj_per_hh_residential_elec_appliances,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2011,35311,2.656550e+07,0.000000e+00,0.000000,752.329330
1,Aruba,ABW,2012,35490,2.666319e+07,0.000000e+00,0.000000,751.287485
2,Aruba,ABW,2013,35697,2.676111e+07,0.000000e+00,0.000000,749.673942
3,Aruba,ABW,2014,35909,2.685855e+07,0.000000e+00,0.000000,747.961548
4,Aruba,ABW,2015,36103,2.695507e+07,0.000000e+00,0.000000,746.615669
...,...,...,...,...,...,...,...,...
5606,Zimbabwe,ZWE,2046,3140103,1.577604e+08,8.833231e+06,2.813039,50.240531
5607,Zimbabwe,ZWE,2047,3134648,1.570312e+08,9.143963e+06,2.917062,50.095323
5608,Zimbabwe,ZWE,2048,3129192,1.563020e+08,9.454694e+06,3.021449,49.949625
5609,Zimbabwe,ZWE,2049,3123737,1.555727e+08,9.765425e+06,3.126200,49.803402


In [24]:
df_hh_heat_elect_all[(df_hh_heat_elect_all['Nation']=='Brazil')&
                    (df_hh_heat_elect_all['Year']==2015)].to_csv('../row_data/hh_heat_elec_gj_BRA_2015.csv')
df_hh_heat_elect_all[(df_hh_heat_elect_all['Nation']=='Brazil')&
                    (df_hh_heat_elect_all['Year']==2015)]

,Nation,iso_code3,Year,hh,Heat_Total_GJ,Elect_Total_GJ,consumpinit_scoe_gj_per_hh_residential_elec_appliances,consumpinit_scoe_gj_per_hh_residential_heat_energy
202,Brazil,BRA,2015,56640376,550420000.0,750190000.0,13.244792,9.717803


In [25]:
# #scoe crowss walk 
# scoe_energy_demands = {
#    consumpinit_scoe_gj_per_hh_residential_elec_appliances : [all but (residential space heating +residential cooking+ clothes dryers)]
#    consumpinit_scoe_gj_per_hh_residential_heat_energy : [residential space heating +residential cooking+ clothes dryers]
#    consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances : (share of electric in residential [0-1.0])*0.9*[sheet: services, End use =="other building energy use" & PRoduct= "total final energy"]
#    consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy  (1- share of electri in residential)*0.9*[sheet: services, End use =="other building energy use" & PRoduct= "total final energy"]
#    consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances :  (share of electric in residential [0-1.0])*0.1*[sheet: services, End use =="other building energy use"]
#    consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy (1-share of electric in residential [0-1.0])*0.1*[sheet: services, End use =="other building energy use"]
total_heat_elect_df['Share_of_electricity']=total_heat_elect_df.apply(lambda row:
                    row['Elect_Total_GJ'] / (row['Heat_Total_GJ']+ row['Elect_Total_GJ']) 
                    if (row['Heat_Total_GJ']+ row['Elect_Total_GJ'] )!=0 else 0, axis=1)
total_heat_elect_df['Share_of_heat']=total_heat_elect_df.apply(lambda row: 1-row['Share_of_electricity'], axis=1)
total_heat_elect_df


,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08,0.226376,0.773624
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08,0.229287,0.770713
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08,0.232084,0.767916
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08,0.234669,0.765331
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08,0.237002,0.762998
...,...,...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06,0.053023,0.946977
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06,0.055026,0.944974
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06,0.057040,0.942960
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06,0.059063,0.940937


In [26]:
df_energy_services=pd.read_excel('../row_data/IEA - Energy Efficiency Indicators database - Extended (December 2022).xlsb',
                        engine='pyxlsb',sheet_name='Services - Energy',skiprows=[0])
df_energy_services=df_energy_services.replace('..',0)
df_energy_bilding=df_energy_services[(df_energy_services['End use']=='Other building energy use')&
                                    (df_energy_services['Product'].str.startswith('Total final energy use (PJ)'))]
df_energy_bilding

,Country,End use,Product,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
33,Australia,Other building energy use,Total final energy use (PJ) ...,212.02,217.68,224.95,254.08,259.77,260.31,263.75,...,297.01,294.91,303.08,310.23,327.90,327.11,334.32,338.31,338.78,333.04
75,Austria,Other building energy use,Total final energy use (PJ) ...,40.00,42.50,41.06,43.16,41.84,43.23,44.11,...,39.08,39.67,40.23,39.84,40.53,38.41,40.53,40.96,41.14,38.31
117,Belgium,Other building energy use,Total final energy use (PJ) ...,145.54,152.85,158.65,167.36,195.08,195.97,190.24,...,184.38,193.25,205.30,177.43,193.01,194.07,194.07,194.50,192.07,185.86
192,Canada,Other building energy use,Total final energy use (PJ) ...,195.00,196.66,205.02,219.90,222.06,213.27,218.24,...,251.58,246.71,259.69,265.83,263.50,265.38,287.93,293.65,300.10,286.62
234,Czech Republic,Other building energy use,Total final energy use (PJ) ...,124.41,134.61,126.82,143.56,144.63,129.94,128.04,...,130.91,126.98,125.69,121.98,124.22,128.92,132.57,130.29,132.94,124.12
309,Denmark,Other building energy use,Total final energy use (PJ) ...,38.92,39.29,40.14,40.00,40.49,40.82,42.05,...,40.23,39.41,39.14,38.74,38.20,38.44,39.89,39.78,39.71,76.29
351,Estonia,Other building energy use,Total final energy use (PJ) ...,12.08,12.42,12.41,14.19,15.94,16.29,16.47,...,16.85,17.75,17.51,19.17,19.50,20.87,19.63,20.54,19.57,19.75
393,Finland,Other building energy use,Total final energy use (PJ) ...,53.20,56.54,58.48,58.72,60.28,61.40,62.99,...,67.68,69.84,68.41,68.02,67.31,65.24,68.21,68.18,66.69,63.14
435,France,Other building energy use,Total final energy use (PJ) ...,160.86,163.94,160.34,164.79,167.78,165.53,165.53,...,165.10,168.47,169.01,174.42,172.50,165.66,170.28,170.52,169.56,175.77
510,Germany,Other building energy use,Total final energy use (PJ) ...,1227.57,385.84,385.99,376.51,375.79,375.34,440.20,...,446.20,446.38,436.91,447.05,480.66,496.04,485.78,463.60,474.70,480.90


In [27]:
#fill present data with mean value
#input all countries with linear regression
#apply shear of elec fraction
def is_number(s):
    try:
        float(s)
        return True
#     except ValueError:
    except:
        return False
nation_mean=[]
for nation in df_energy_bilding['Country'].unique():
    values=[val for val in df_energy_bilding.loc[df_energy_bilding['Country']==nation].values[0] if is_number(val)]
    nation_mean.append([nation,np.mean(values)])
mean_building=pd.DataFrame(nation_mean,columns=['Nation','building_energy_mean'])
mean_building


,Nation,building_energy_mean
0,Australia,287.163333
1,Austria,40.919524
2,Belgium,186.136190
3,Canada,244.880476
4,Czech Republic,129.657143
5,Denmark,41.780000
6,Estonia,17.199524
7,Finland,64.332381
8,France,168.476667
9,Germany,477.399048


In [28]:
coun_year_building=[]
for c in df_energy_bilding['Country'].unique():
    for y in years_hist:
        building_data=df_energy_bilding[df_energy_bilding['Country']==c][y].values[0]
        if building_data==0:
            building_data=mean_building[mean_building['Nation']==c]['building_energy_mean'].values[0]
        coun_year_building.append([c,y,building_data])

building_complete_hist=pd.DataFrame(coun_year_building,columns=['Nation','Year','Other building energy use'])
        
building_complete_hist

,Nation,Year,Other building energy use
0,Australia,2011,297.01
1,Australia,2012,294.91
2,Australia,2013,303.08
3,Australia,2014,310.23
4,Australia,2015,327.90
...,...,...,...
508,Uruguay,2015,7.50
509,Uruguay,2016,7.86
510,Uruguay,2017,7.66
511,Uruguay,2018,8.06


In [29]:
building_pop_gdp=building_complete_hist.merge(df_pop_gdp_all,on=['Nation','Year'])
building_pop_gdp

,Nation,Year,Other building energy use,iso_code3,population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions)
0,Australia,2011,297.01,AUS,1.905604e+07,3.283984e+06,861.792237,19.056040,3.283984
1,Australia,2012,294.91,AUS,1.941483e+07,3.318631e+06,895.123477,19.414834,3.318631
2,Australia,2013,303.08,AUS,1.977501e+07,3.353116e+06,928.454717,19.775013,3.353116
3,Australia,2014,310.23,AUS,2.009566e+07,3.380029e+06,961.785957,20.095657,3.380029
4,Australia,2015,327.90,AUS,2.041055e+07,3.405449e+06,995.117197,20.410546,3.405449
...,...,...,...,...,...,...,...,...,...
445,Uruguay,2015,7.50,URY,3.242948e+06,1.690652e+05,51.626567,3.242948,0.169065
446,Uruguay,2016,7.86,URY,3.257863e+06,1.662762e+05,52.159029,3.257863,0.166276
447,Uruguay,2017,7.66,URY,3.273061e+06,1.635843e+05,52.691490,3.273061,0.163584
448,Uruguay,2018,8.06,URY,3.288346e+06,1.609439e+05,53.223952,3.288346,0.160944


In [30]:
#input all countries with linear regression
X=building_pop_gdp[['population_gnrl_urban(millions)','population_gnrl_rural(millions)','gdp_mmm_usd']]
Y=building_pop_gdp['Other building energy use']

linear_model_building = LinearRegression()

linear_model_building.fit(X,Y)

df_building_all=df_pop_gdp_all.copy()

for c in df_pop_gdp_all['Nation'].unique():
    for y in years_all:
        if (c,y) in zip(building_pop_gdp['Nation'],building_pop_gdp['Year']):
            
            row=building_pop_gdp[(building_pop_gdp['Nation']==c)&(building_pop_gdp['Year']==y)]
            build_data=row['Other building energy use'].values[0]*1e3

        else:
            
            X_test = df_pop_gdp_all.loc[(df_pop_gdp_all['Nation'] == c) & 
                                (df_pop_gdp_all['Year']== y)][['population_gnrl_urban(millions)',
                                                                  'population_gnrl_rural(millions)',
                                                                  'gdp_mmm_usd']]
            
            build_data=linear_model_building.predict(X_test)[0]*1e3
            
            if build_data<0:
                build_data=0
                
        total_heat_elect_df.loc[(total_heat_elect_df['Nation']==c)&
                            (total_heat_elect_df['Year']==y),'Other building energy use(TJ)']=build_data

total_heat_elect_df

,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat,Other building energy use(TJ)
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08,0.226376,0.773624,0.0
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08,0.229287,0.770713,0.0
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08,0.232084,0.767916,0.0
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08,0.234669,0.765331,0.0
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08,0.237002,0.762998,0.0
...,...,...,...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06,0.053023,0.946977,0.0
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06,0.055026,0.944974,0.0
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06,0.057040,0.942960,0.0
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06,0.059063,0.940937,0.0


In [31]:
# consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances : (share of electric in residential [0-1.0])*0.9*[sheet: services, End use =="other building energy use" & PRoduct= "total final energy"]
#    consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy  (1- share of electri in residential)*0.9*[sheet: services, End use =="other building energy use" & PRoduct= "total final energy"]
#    consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances :  (share of electric in residential [0-1.0])*0.1*[sheet: services, End use =="other building energy use"]
#    consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy (1-share of electric in residential [0-1.0])*0.1*[sheet: services, End use =="other building energy use"]
total_heat_elect_df=total_heat_elect_df.merge(df_pop_gdp_all, on=['Nation','iso_code3','Year'])

total_heat_elect_df

,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat,Other building energy use(TJ),population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions)
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08,0.226376,0.773624,0.0,7.212518e+06,2.290489e+07,35.369178,7.212518,22.904893
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08,0.229287,0.770713,0.0,7.528589e+06,2.363279e+07,38.085386,7.528589,23.632789
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08,0.232084,0.767916,0.0,7.865068e+06,2.440452e+07,40.801594,7.865068,24.404524
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08,0.234669,0.765331,0.0,8.204880e+06,2.516592e+07,43.517802,8.204880,25.165924
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08,0.237002,0.762998,0.0,8.535606e+06,2.587800e+07,46.234010,8.535606,25.877997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06,0.053023,0.946977,0.0,5.352032e+06,7.863317e+06,24.382002,5.352032,7.863317
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06,0.055026,0.944974,0.0,5.384492e+06,7.807897e+06,26.002852,5.384492,7.807897
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06,0.057040,0.942960,0.0,5.416953e+06,7.752477e+06,27.623703,5.416953,7.752477
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06,0.059063,0.940937,0.0,5.449414e+06,7.697057e+06,29.244553,5.449414,7.697057


In [32]:

total_heat_elect_df['consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances']=total_heat_elect_df.apply(
    lambda row: row['Share_of_electricity'] *0.9 * row['Other building energy use(TJ)']/row['gdp_mmm_usd'], axis=1) 

total_heat_elect_df['consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy']=total_heat_elect_df.apply(
    lambda row: row['Share_of_heat'] *0.9 * row['Other building energy use(TJ)']/row['gdp_mmm_usd'], axis=1) 

total_heat_elect_df['consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances']=total_heat_elect_df.apply(
    lambda row: row['Share_of_electricity'] *0.1 * row['Other building energy use(TJ)']/row['gdp_mmm_usd'], axis=1) 

total_heat_elect_df['consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy']=total_heat_elect_df.apply(
    lambda row: row['Share_of_heat'] *0.1 * row['Other building energy use(TJ)']/row['gdp_mmm_usd'], axis=1) 


total_heat_elect_df


,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat,Other building energy use(TJ),population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions),consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy,consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08,0.226376,0.773624,0.0,7.212518e+06,2.290489e+07,35.369178,7.212518,22.904893,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08,0.229287,0.770713,0.0,7.528589e+06,2.363279e+07,38.085386,7.528589,23.632789,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08,0.232084,0.767916,0.0,7.865068e+06,2.440452e+07,40.801594,7.865068,24.404524,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08,0.234669,0.765331,0.0,8.204880e+06,2.516592e+07,43.517802,8.204880,25.165924,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08,0.237002,0.762998,0.0,8.535606e+06,2.587800e+07,46.234010,8.535606,25.877997,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06,0.053023,0.946977,0.0,5.352032e+06,7.863317e+06,24.382002,5.352032,7.863317,0.0,0.0,0.0,0.0
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06,0.055026,0.944974,0.0,5.384492e+06,7.807897e+06,26.002852,5.384492,7.807897,0.0,0.0,0.0,0.0
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06,0.057040,0.942960,0.0,5.416953e+06,7.752477e+06,27.623703,5.416953,7.752477,0.0,0.0,0.0,0.0
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06,0.059063,0.940937,0.0,5.449414e+06,7.697057e+06,29.244553,5.449414,7.697057,0.0,0.0,0.0,0.0


In [33]:
total_heat_elect_df_hist=total_heat_elect_df[total_heat_elect_df['Year'] < 2020]
total_heat_elect_df_proj=total_heat_elect_df[total_heat_elect_df['Year'] >= 2020]

total_heat_elect_df_hist

,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat,Other building energy use(TJ),population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions),consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy,consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy
0,Afghanistan,AFG,2011,4.687038e+08,1.371512e+08,0.226376,0.773624,0.0,7.212518e+06,2.290489e+07,35.369178,7.212518,22.904893,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2012,4.828102e+08,1.436363e+08,0.229287,0.770713,0.0,7.528589e+06,2.363279e+07,38.085386,7.528589,23.632789,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2013,4.976805e+08,1.504121e+08,0.232084,0.767916,0.0,7.865068e+06,2.440452e+07,40.801594,7.865068,24.404524,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2014,5.123012e+08,1.570840e+08,0.234669,0.765331,0.0,8.204880e+06,2.516592e+07,43.517802,8.204880,25.165924,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2015,5.259440e+08,1.633688e+08,0.237002,0.762998,0.0,8.535606e+06,2.587800e+07,46.234010,8.535606,25.877997,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7204,Zimbabwe,ZWE,2015,1.888983e+08,1.530004e+07,0.074927,0.925073,0.0,4.473872e+06,9.340770e+06,6.769741,4.473872,9.340770,0.0,0.0,0.0,0.0
7205,Zimbabwe,ZWE,2016,1.917904e+08,1.641311e+07,0.078832,0.921168,0.0,4.531238e+06,9.499100e+06,6.756399,4.531238,9.499100,0.0,0.0,0.0,0.0
7206,Zimbabwe,ZWE,2017,1.944553e+08,1.743358e+07,0.082277,0.917723,0.0,4.589452e+06,9.647147e+06,6.743057,4.589452,9.647147,0.0,0.0,0.0,0.0
7207,Zimbabwe,ZWE,2018,1.969458e+08,1.838103e+07,0.085363,0.914637,0.0,4.650597e+06,9.788215e+06,6.729715,4.650597,9.788215,0.0,0.0,0.0,0.0


In [34]:
total_heat_elect_df_proj

,Nation,iso_code3,Year,Heat_Total_GJ,Elect_Total_GJ,Share_of_electricity,Share_of_heat,Other building energy use(TJ),population_gnrl_urban,population_gnrl_rural,gdp_mmm_usd,population_gnrl_urban(millions),population_gnrl_rural(millions),consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy,consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy
9,Afghanistan,AFG,2020,5.765038e+08,1.838898e+08,0.241835,0.758165,0.0,1.013149e+07,2.879685e+07,50.050108,10.131490,28.796851,0.0,0.0,0.0,0.0
10,Afghanistan,AFG,2021,6.113884e+08,1.979817e+08,0.244612,0.755388,0.0,1.095378e+07,3.071641e+07,51.885013,10.953783,30.716406,0.0,0.0,0.0,0.0
11,Afghanistan,AFG,2022,6.234978e+08,2.032529e+08,0.245845,0.754155,0.0,1.134045e+07,3.139778e+07,53.719919,11.340453,31.397784,0.0,0.0,0.0,0.0
12,Afghanistan,AFG,2023,6.356073e+08,2.085241e+08,0.247028,0.752972,0.0,1.172712e+07,3.207916e+07,55.554825,11.727123,32.079162,0.0,0.0,0.0,0.0
13,Afghanistan,AFG,2024,6.477167e+08,2.137954e+08,0.248163,0.751837,0.0,1.211379e+07,3.276054e+07,57.389731,12.113792,32.760540,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7235,Zimbabwe,ZWE,2046,1.577604e+08,8.833231e+06,0.053023,0.946977,0.0,5.352032e+06,7.863317e+06,24.382002,5.352032,7.863317,0.0,0.0,0.0,0.0
7236,Zimbabwe,ZWE,2047,1.570312e+08,9.143963e+06,0.055026,0.944974,0.0,5.384492e+06,7.807897e+06,26.002852,5.384492,7.807897,0.0,0.0,0.0,0.0
7237,Zimbabwe,ZWE,2048,1.563020e+08,9.454694e+06,0.057040,0.942960,0.0,5.416953e+06,7.752477e+06,27.623703,5.416953,7.752477,0.0,0.0,0.0,0.0
7238,Zimbabwe,ZWE,2049,1.555727e+08,9.765425e+06,0.059063,0.940937,0.0,5.449414e+06,7.697057e+06,29.244553,5.449414,7.697057,0.0,0.0,0.0,0.0


In [35]:
for ssp_var in ['consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances',
               'consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy',
                'consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances',
                'consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy']:
    

    total_heat_elect_df_hist[['Nation','iso_code3','Year',
    ssp_var]].to_csv(grandparentdir+'/'+ssp_var+
    '/input_to_sisepuede/historical/'+ssp_var+'.csv')

    total_heat_elect_df_proj[['Nation','iso_code3','Year',
    ssp_var]].to_csv(grandparentdir+'/'+ssp_var+
    '/input_to_sisepuede/projected/'+ssp_var+'.csv')
    
    total_heat_elect_df_hist[['Nation','iso_code3','Year',
    ssp_var]].loc[(total_heat_elect_df_hist['Nation']=='Brazil')&
                 (total_heat_elect_df_hist['Year']==2015)].to_csv(grandparentdir+'/'+ssp_var+
    '/row_data/'+ssp_var+'_Bra_2015.csv')
    total_heat_elect_df.to_csv(grandparentdir+'/'+ssp_var+
    '/row_data/'+ssp_var[0:30]+'_all.csv')



In [36]:
total_heat_elect_df_hist[['Nation','iso_code3','Year',
    ssp_var]].loc[(total_heat_elect_df_hist['Nation']=='Brazil')&
                 (total_heat_elect_df_hist['Year']==2015)]

,Nation,iso_code3,Year,consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy
884,Brazil,BRA,2015,8.286033
